In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix

## 資料讀取
讀入原方法之：
* accuracy
* prediction
以及原資料之真實結果

In [2]:
import pickle
PIK = 'Answer.dat'
with open(PIK, 'rb') as f:
    file = pickle.load(f)

In [3]:
PIK = 'testAnswer.dat'
with open(PIK, 'rb') as f:
    testfile = pickle.load(f)


In [4]:
data = pd.DataFrame.from_csv("test.csv")
print("Original Length: ", len(data))
data = data.dropna(axis=0)
print("Dropna Length: ", len(data))
data.head(1)

Original Length:  6305988
Dropna Length:  5621365


,Primary Type,Description,Location Description,Arrest,Domestic,Ward,Community Area,Year,Latitude,Longitude,distance,Month,Day,Hour
ID,,,,,,,,,,,,,,
10404989,WEAPONS VIOLATION,UNLAWFUL POSS OF HANDGUN,STREET,True,False,28.0,25.0,2016,41.888659,-87.748942,0.00168,2,5,11


In [5]:
#Cut off outliers
data = data[data['Latitude'] > 41.644]
data = data[data['Latitude'] < 42.020]
data = data[data['Longitude'] > -87.940]
data = data[data['Longitude'] < -87.521]
data = data[data['distance'] < 100000]
print("Cut off outliers:", len(data))

Cut off outliers: 5602755


In [6]:
data = data.rename(columns={"Location Description": "LocationDescription"})
data = data.rename(columns={"Primary Type": "PrimaryType"})
data_test = data[data['Year']==2017]
data1 = data[data['Year'] < 2017]

## adaboost
對accuracy做adaboost  
挑出分類效果夠好的"Primary Type"和"Description"  
以及去除分類效果最差的"Domestic"  
得到個欄位的權重

In [7]:
train = data1['Arrest']
eps = []
alpha = []
D = []
D.append([1.0/len(file[0])]*len(file[0]))
count=-1
for case in file:
    count=count+1
    if not count in [2,4,5,6,7,8]:
        alpha.append(0.0)
        continue
    index_e = np.where(train != case)[0]
    index_c = np.where(train == case)[0]
    eps.append(0)
    for i in index_e:
        eps[-1]=eps[-1]+D[-1][int(i)]
    alpha.append(0.5*np.log((1-eps[-1])/eps[-1]))
    D.append(D[-1])
    for i in index_e:
        D[-1][int(i)] = D[-1][int(i)]*np.exp(alpha[-1])
    for i in index_c:
        D[-1][int(i)] = D[-1][int(i)]*np.exp(-alpha[-1])
    D[-1] = D[-1]/sum(D[-1])
    
print(len(eps))
print(len(alpha))
print(len(D))
print(len(file))
print(alpha)

6
9
7
9
[0.0, 0.0, 0.17854670738750478, 0.0, 0.20575879913772321, 0.14802665333947249, -0.23353065084026622, -0.10704361143559446, -0.07358669172533297]


## 將原類別結果改寫成數值狀態

In [8]:
for col in range(len(testfile)):
    for row in range(len(testfile[col])):
        if testfile[col][row]:
            testfile[col][row]=1.0
        else:
            testfile[col][row]=-1.0

In [9]:
for col in range(len(file)):
    for row in range(len(file[col])):
        if file[col][row]:
            file[col][row]=1.0
        else:
            file[col][row]=-1.0

## 代入權重
各欄位結果加權後的總合  
大於0視為True  
小於等於0視作False
計算新的accuracy

In [10]:
adaboost_train = [0.0]*len(train)
for i in range(len(file)):
    for j in range(len(file[i])):
        adaboost_train[j]= adaboost_train[j]+file[i][j]*alpha[i]
for i in range(len(adaboost_train)):
    if adaboost_train[i]<=0:
        adaboost_train[i] = False
    else:
        adaboost_train[i] = True
print("AC: ",len(np.where(adaboost_train==train)[0])/len(train))
print(confusion_matrix(train, adaboost_train,labels=[True, False]))

AC:  0.6657088946871167
[[ 587172  980623]
 [ 872377 3102901]]


同樣對prediction操作得到新的預測

In [13]:
adaboost_ans = [0.0]*len(testfile[0])
for i in range(len(testfile)):
    for j in range(len(testfile[i])):
        adaboost_ans[j]= adaboost_ans[j]+testfile[i][j]*alpha[i]
for i in range(len(adaboost_ans)):
    if adaboost_ans[i]<=0:
        adaboost_ans[i] = False
    else:
        adaboost_ans[i] = True
print("AC: ",len(np.where(adaboost_ans==test)[0])/len(test))
print(confusion_matrix(test, adaboost_ans,labels=[True, False]))

AC:  0.7112864850373647
[[ 3823  7843]
 [ 9388 38628]]


## voting base method

將新的accuracy和原accuracy高的欄位  
依權重加權投票   
計算最後accuracy  

In [16]:
adbv =[]
for item in adaboost_ans:
    if item:
        adbv.append(1.0)
    else:
        adbv.append(-1.0)
        
ensemble = [0.0]*len(adbv)
for i in range(len(ensemble)):
    ensemble[i] = adbv[i]*0.66571+testfile[0][i]*0.87050+testfile[1][i]*0.86964
    if ensemble[i] <= 0:
        ensemble[i] = False
    else:
        ensemble[i] = True
print("AC: ",len(np.where(ensemble==test)[0])/len(test))
print(confusion_matrix(test, ensemble,labels=[True, False]))

AC:  0.8774672430548575
[[ 5976  5690]
 [ 1623 46393]]


總結所有方法所得到的預測結果

In [17]:
adbv =[]
for item in adaboost_train:
    if item:
        adbv.append(1.0)
    else:
        adbv.append(-1.0)
        
ensemble = [0.0]*len(adbv)
for i in range(len(ensemble)):
    ensemble[i] = adbv[i]*0.66571+file[0][i]*0.87050+file[1][i]*0.86964
    if ensemble[i] <= 0:
        ensemble[i] = False
    else:
        ensemble[i] = True
print("AC: ",len(np.where(ensemble==train)[0])/len(train))
print(confusion_matrix(train, ensemble,labels=[True, False]))

AC:  0.869820765485138
[[ 959669  608126]
 [ 113467 3861811]]
